## Шаг 1. Загрузка данных

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

data = pd.read_csv('/datasets/data.csv')
display(data)
data.info()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**

Данные в некоторых столбцах представлены не корректно:
days_employed - есть отрицательные значения;
education - значения записаны с больших и маленьких букв;
purpose - требует доработки.
С типами данных на первый взгляд все нормально, трудовой стаж лучше было бы перевести в int.

## Шаг 2. Предобработка данных

In [2]:
data.isna().sum()
#data.isna().mean()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [3]:
#print(data['days_employed'].value_counts()) 
print(data['total_income'].value_counts()) 

# Вывод данных с пропусками и посчитаем их количество
nan_day = data[data['days_employed'].isna()]
nan_income = data[data['total_income'].isna()]
#print(data['days_employed'].value_counts()) 

112874.418757    1
133912.272223    1
182036.676828    1
122421.963500    1
198271.837248    1
                ..
133299.194693    1
115080.782380    1
84896.781597     1
153838.839212    1
150014.128510    1
Name: total_income, Length: 19351, dtype: int64


In [5]:
# Создаем копию датафрейма, чтобы при необходимости в дальнейшем можно было сравнить данные с изначальными.
clear_data = data 

In [6]:
# Убираю отрицательные значения из колонок
clear_data.loc[clear_data['days_employed'] < 0, 'days_employed'] *= -1

#проверим количество полей со стажем более 50 лет (18 250 дней)
#print(clear_data.loc[clear_data['days_employed'] > 18250, 'days_employed'].count())

#сгрупируем данные и посчитаем среднее и медиану для зароботка
count_employed = clear_data.groupby('income_type')['days_employed'].count()
median_employed = clear_data.groupby('income_type')['days_employed'].median()  
 бы хорошо узнать почему такой стаж, но не у кого

clear_data.drop(clear_data[clear_data['income_type'] == 'безработный'].index, inplace=True)
clear_data.drop(clear_data[clear_data['income_type'] == 'в декрете'].index, inplace=True)
clear_data.drop(clear_data[clear_data['income_type'] == 'предприниматель'].index, inplace=True)
clear_data.drop(clear_data[clear_data['income_type'] == 'студент'].index, inplace=True)
# появилось предположение что стаж пенсионеров указан в часах, переведем в дни
#(теперь пенсионеры в среднем имеют сташ 40лет)
clear_data.loc[clear_data['days_employed'] > 18250, 'days_employed'] /= 24


count_employed = clear_data.groupby('income_type')['days_employed'].count()
median_employed = clear_data.groupby('income_type')['days_employed'].median() 
#print(count_employed)
print(median_employed)

income_type
госслужащий     2689.368353
компаньон       1547.382223
пенсионер      15217.221094
сотрудник       1573.956856
Name: days_employed, dtype: float64


In [7]:
#посмотрим данные из total_incom
display(clear_data.head())
#print(clear_data.loc[clear_data['days_employed'] < 0, 'days_employed'].count())

# Отрицательных значений нет, посмотрим среднее от образования
clear_data['education'] = clear_data['education'].str.lower() 
#print(clear_data.groupby('education')['total_income'].median())

#Видим что с данными все в порядке и есть зависимость от уровня образования

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,14177.753002,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [8]:
# Пропущенные данные из  total_income заменим средними относительно образования
clear_data.loc[clear_data['total_income'].isnull(),'total_income'] = clear_data['education']

clear_data.loc[clear_data['total_income'] == 'среднее', 'total_income'] = 136487.432242
clear_data.loc[clear_data['total_income'] == 'высшее', 'total_income'] = 175270.760792
clear_data.loc[clear_data['total_income'] == 'неоконченное высшее', 'total_income'] = 160115.398644
clear_data.loc[clear_data['total_income'] == 'начальное', 'total_income'] = 117137.352825
clear_data.loc[clear_data['total_income'] == 'ученая степень', 'total_income'] = 157259.898555

print(clear_data['total_income'].value_counts())
data.isna().sum()
#print(median_income)

136487.432242    1540
175270.760792     543
160115.398644      69
117137.352825      21
141949.576902       1
                 ... 
120191.530490       1
87425.716404        1
87426.721962        1
87427.652188        1
81054.288607        1
Name: total_income, Length: 19350, dtype: int64


children               0
days_employed       2173
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
dtype: int64

In [9]:
# Пришло время разобраться со стажем, пропущенные данные из  days_employed заменим средними относительно должности
clear_data.loc[clear_data['days_employed'].isnull(),'days_employed'] = clear_data['income_type']

clear_data.loc[clear_data['days_employed'] == 'госслужащий', 'days_employed'] = 2689.368353
clear_data.loc[clear_data['days_employed'] == 'компаньон', 'days_employed'] = 1547.382223
clear_data.loc[clear_data['days_employed'] == 'пенсионер', 'days_employed'] = 15217.221094
clear_data.loc[clear_data['days_employed'] == 'сотрудник', 'days_employed'] = 1573.956856

print(clear_data['days_employed'].value_counts())
#data.isna().sum()
#print(median_income)

1573.956856     1105
1547.382223      508
15217.221094     413
2689.368353      147
7927.550380        1
                ... 
370.868941         1
226.256392         1
454.632610         1
118.216085         1
1007.078931        1
Name: days_employed, Length: 19350, dtype: int64


**Вывод**

<li>определите и заполните пропущенные значения:</li>
<ul>
  <li>найдены пропуски Nan в столбцах 'days_employed'  и 'total_income';</li>
  <li>пропуски могли возникнуть из-за сбоев(при предаче данных), возможно это определеная котегория клиентов, или клиенты не захотели предоставить данные;</li>
  <li>Пропуски заполнены средним значением по колонке в зависемости от образования или должности; </li>
</ul>    

### Замена типа данных

In [10]:
# Замена типа данных на более удобный и округление.
clear_data['days_employed'] = clear_data['days_employed'].astype(float)
clear_data['days_employed'] = np.round(clear_data['days_employed']).astype('int')
clear_data['total_income'] = clear_data['total_income'].astype(float)
clear_data['total_income'] = np.round(clear_data['total_income']).astype('int')
display(clear_data.head())
#clear_data['gender'].unique()
#clear_data[clear_data['gender'] == 'XNA'].sum()
clear_data.info()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8438,42,высшее,0,женат / замужем,0,F,сотрудник,0,253876,покупка жилья
1,1,4025,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145886,покупка жилья
3,3,4125,32,среднее,1,женат / замужем,0,M,сотрудник,0,267629,дополнительное образование
4,0,14178,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


<class 'pandas.core.frame.DataFrame'>
Int64Index: 21519 entries, 0 to 21524
Data columns (total 12 columns):
children            21519 non-null int64
days_employed       21519 non-null int64
dob_years           21519 non-null int64
education           21519 non-null object
education_id        21519 non-null int64
family_status       21519 non-null object
family_status_id    21519 non-null int64
gender              21519 non-null object
income_type         21519 non-null object
debt                21519 non-null int64
total_income        21519 non-null int64
purpose             21519 non-null object
dtypes: int64(7), object(5)
memory usage: 2.8+ MB


### Обработка дубликатов

In [11]:
# общий поиск дупликатов
display(clear_data.duplicated().sum())
# так как ранее было обнаружено что значения пишутся в разном регистре упорядочим их, приведя к нижнему регистру.
clear_data['education'] = clear_data['education'].str.lower() 
clear_data['family_status'] = clear_data['family_status'].str.lower() 
#Удалим дупликаты так как их число не значительно
clear_data = clear_data.drop_duplicates().reset_index(drop= True)
display(clear_data.duplicated().sum())
#display(clear_data.head())

71

0

**Вывод**

Так как датафрейм большой были использованы методы для поиска и удаления дубликатов, одной из возможных причин их возникновения может быть обращение клиента несколько раз.

### Лемматизация

In [12]:
#clear_data['purpose'].unique()
# pymystem3 импортируем библиотекb:
from pymystem3 import Mystem
m = Mystem() 
from collections import Counter

# проводим леметизацию по purpose
lem_categ = []
def lem(text):
    lemmas = m.lemmatize(text) 
    lem_categ.append(' '.join(lemmas)) 
    return lemmas

clear_data['lem_purpose'] = clear_data['purpose'].apply(lem)
display(clear_data.head())
print(Counter(lem_categ))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lem_purpose
0,1,8438,42,высшее,0,женат / замужем,0,F,сотрудник,0,253876,покупка жилья,"[покупка, , жилье, \n]"
1,1,4025,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]"
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145886,покупка жилья,"[покупка, , жилье, \n]"
3,3,4125,32,среднее,1,женат / замужем,0,M,сотрудник,0,267629,дополнительное образование,"[дополнительный, , образование, \n]"
4,0,14178,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]"


Counter({'автомобиль \n': 971, 'свадьба \n': 791, 'на   проведение   свадьба \n': 767, 'сыграть   свадьба \n': 765, 'операция   с   недвижимость \n': 675, 'покупка   коммерческий   недвижимость \n': 661, 'операция   с   жилье \n': 652, 'операция   с   коммерческий   недвижимость \n': 650, 'покупка   жилье   для   сдача \n': 650, 'покупка   жилье \n': 646, 'жилье \n': 646, 'покупка   жилье   для   семья \n': 638, 'строительство   собственный   недвижимость \n': 634, 'недвижимость \n': 633, 'операция   со   свой   недвижимость \n': 627, 'строительство   жилой   недвижимость \n': 624, 'покупка   недвижимость \n': 621, 'покупка   свой   жилье \n': 620, 'строительство   недвижимость \n': 619, 'ремонт   жилье \n': 606, 'покупка   жилой   недвижимость \n': 605, 'на   покупка   свой   автомобиль \n': 505, 'заниматься   высокий   образование \n': 496, 'сделка   с   подержанный   автомобиль \n': 486, 'на   покупка   подержать   автомобиль \n': 478, 'свой   автомобиль \n': 478, 'на   покупка   ав

**Вывод**

При расмотрении столбца purpose датафрейма видно, что много похожих методов описания цели кредита, лемматизация помогла выделить основы слов для дальнейшей категоризации.

### Категоризация данных

In [13]:
#категоризация дохода
def total_income(pay):
    """
    Возвращает уровень дохода:
    - 'низкий доход', если pay <= 100000;
    - 'средний доход', если pay более 100000 и менее 250000, включая 100000 и 250000;
    - 'высокий доход' — во всех остальных случаях.
    """    
    if pay <= 100000:
        return 'низкий доход'
    if pay <= 200000:
        return 'средний доход'
    return 'высокий доход'

clear_data['pay_category'] = clear_data['total_income'].apply(total_income)

#категоризация стажа
def stage(day):
    """
    Возвращает стаж:
    - 'маленький стаж', если вay <= 1 095 дней;
    - 'средний стаж', если вay более 1 095 и менее 3 650, включая 1 095 и 3 650;
    - 'большой стаж' — во всех остальных случаях.
    - 'аномали' стаж более 50 лет 
    """    
    if day <= 1095:
        return 'маленький стаж'
    if day <= 3650:
        return 'средний стаж'
    if day <= 18250:
        return 'большой стаж'
    return 'аномалия' 

clear_data['stage_category'] = clear_data['days_employed'].apply(stage)

#категоризация лем
def lem(lems):
    """
    Возвращает очищенные строки:
    Недвижимость можно поделить на несколько категорий
    """    
    if 'жилье' in lems:
        return 'недвижимости'
    if 'недвижимость' in lems:
        return 'недвижимости'
    if 'свадьба' in lems:
        return 'свадьба'
    if 'автомобиль' in lems:
        return 'автомобиль' 
    if 'образование' in lems:
        return 'образование'
    return 'такого не найдено'

clear_data['lem_category'] = clear_data['lem_purpose'].apply(lem)
display(clear_data.head())
#Ловим все категории
#clear_data['lem_category'].unique()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lem_purpose,pay_category,stage_category,lem_category
0,1,8438,42,высшее,0,женат / замужем,0,F,сотрудник,0,253876,покупка жилья,"[покупка, , жилье, \n]",высокий доход,большой стаж,недвижимости
1,1,4025,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]",средний доход,большой стаж,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145886,покупка жилья,"[покупка, , жилье, \n]",средний доход,большой стаж,недвижимости
3,3,4125,32,среднее,1,женат / замужем,0,M,сотрудник,0,267629,дополнительное образование,"[дополнительный, , образование, \n]",высокий доход,большой стаж,образование
4,0,14178,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]",средний доход,большой стаж,свадьба


**Вывод**

Для категоризаци выбраны данные которые сложно обрабатывать по отдельности, они сгрупированы в категории для дальнейшего анализа.

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [14]:
#clear_data['children'].corr(clear_data['debt'])

#Для начала проверим какие значения есть в столбце дети и возврат долгов
clear_data['children'].unique()
#clear_data['debt'].unique()

# в столбце 'children' есть сомнительные данные -1 и 20 посчитаем их колмчество
count_child = clear_data[clear_data['children'] == -1]['children'].count()
count_child += clear_data[clear_data['children'] == 20]['children'].count()
print(count_child)

# всего 123 значения думаю можно присвоить 0 
clear_data.loc[(clear_data['children'] == -1)] = 0
clear_data.loc[(clear_data['children'] == 20)] = 0
clear_data['children'].unique()

# теперь можно свести все данные в таблицу и посмотреть 
depend_child = pd.pivot_table(clear_data, values='debt', index='children', aggfunc=['count', 'sum'])
depend_child.columns = ['all_count', 'not_debt']
depend_child['depend_child_prsent'] = depend_child['not_debt'] / depend_child['all_count']
display(depend_child)

123


,all_count,not_debt,depend_child_prsent
children,,,
0,14210,1063,0.074806
1,4807,443,0.092157
2,2051,193,0.094100
3,330,27,0.081818
4,41,4,0.097561
5,9,0,0.000000


**Вывод**

можно сказать следующее - наличие детей в семье оказывает незначительное влияние на возврат долга, среди тех у кого нет детей на 2% меньше невозвратов, а так же можно отследить зависимость, чем больше детей тем меньше берут кредит.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [15]:
# cведем все данные в таблицу и посмотрим

depend_family_status = pd.pivot_table(clear_data, values='debt', index='family_status', aggfunc=['count', 'sum'])
depend_family_status.columns = ['all_count', 'not_debt']
depend_family_status['depend_family_status_prsent'] = depend_family_status['not_debt'] / depend_family_status['all_count']
display(depend_family_status)

,all_count,not_debt,depend_family_status_prsent
family_status,,,
0,123,0,0.000000
в разводе,1189,84,0.070648
вдовец / вдова,951,63,0.066246
гражданский брак,4132,385,0.093175
женат / замужем,12258,925,0.075461
не женат / не замужем,2795,273,0.097674


**Вывод**

Семейное положение тоже оказывает влияние, больше всего невозвратов у тех кто живет гражданским браком и холост.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [16]:
#clear_data['total_income'].corr(clear_data['debt'])
depend_pay_category = pd.pivot_table(clear_data, values='debt', index='pay_category', aggfunc=['count', 'sum'])
depend_pay_category.columns = ['all_count', 'not_debt']
depend_pay_category['depend_pay_category_prsent'] = depend_pay_category['not_debt'] / depend_pay_category['all_count']
#depend_child_prsent.rename(columns={'children': 'persent'})
display(depend_pay_category)

,all_count,not_debt,depend_pay_category_prsent
pay_category,,,
0,123,0,0.000000
высокий доход,5036,356,0.070691
низкий доход,4441,352,0.079261
средний доход,11848,1022,0.086259


**Вывод**

В категории по доходам чаще всего не возвращают люди с средним доходом, у тех кто имеет высокий доход процент невазврата ниже всего.

- Как разные цели кредита влияют на его возврат в срок?

In [17]:
depend_lem_category = pd.pivot_table(clear_data, values='debt', index='lem_category', aggfunc=['count', 'sum'])
depend_lem_category.columns = ['all_count', 'not_debt']
depend_lem_category['depend_lem_category_prsent'] = depend_lem_category['not_debt'] / depend_lem_category['all_count']
#depend_child_prsent.rename(columns={'children': 'persent'})
display(depend_lem_category)

,all_count,not_debt,depend_lem_category_prsent
lem_category,,,
0,123,0,0.000000
автомобиль,4278,399,0.093268
недвижимости,10747,779,0.072485
образование,3988,369,0.092528
свадьба,2312,183,0.079152


**Вывод**

Сами рисковаными кредитами оказались кредиты на образование и автомобили, тут количество невозвратов чуть более 9%.

## Шаг 4. Общий вывод

<li>Подводя итоги можно сказать следующее - наличие детей в семье оказывает незначительное влияние на возврат долга, среди тех у кого нет детей на 2% меньше невозвратов, а так же можно отследить зависимость, чем больше детей тем меньше берут кредит.</li>
<li>Семейное положение тоже оказывает влияние, больше всего невозвратов у тех кто живет гражданским браком и холост.</li>
<li>В категории по доходам чаще всего не возвращают люди с средним доходом, у тех кто имеет высокий доход процент невазврата ниже всего.</li>
<li>Самими рисковаными кредитами оказались кредиты на образование и автомобили, тут количество невозвратов чуть более 9%.</li>
<li>При анализе данных были найдены пропуски, сначало было принято решение удалить все пропуски, но при подсчете выяснилось ,что их около 10%, а это уже может повлиять на общий результат, тогда было принято решение земенить их на медианные значения, но и тут проанализировав результаты стало понятно, что данные могут быть связаны с другими колонками, тогда было решено, заменять средним по категориям от которых могут зависить пропуски.</li>